# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
 # Import the city csv
data = pd.read_csv("../output_data/cities.csv", dtype="object", encoding="utf-8")

# Visualize
data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

 # Store 'Humidity' in  Humidity 
humidity = data["Humidity"].astype(float)
maxhumidity = humidity.max()

 # Store 'Lat' and 'Lng' into  locations 
locations = data[["Lat", "Lng"]].astype(float)

In [5]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
max_intensity

290.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
cities_narrowed_df = data.dropna()

cities_narrowed_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Store into variable named hotel_df
hotel_df = cities_narrowed_df.loc[:,["City", "Country", "Lat", "Lng"]] 

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Visualize hotel data frame

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.7,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,


In [9]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location": "Lat", "Lng"
    "keyword": "hotel",
    "type": "hotel",
    "radius": 5000,
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel type from df
    city_name = row['City']
    lat = row["Lat"]
    lng = row["Lng"]

    # add keyword to params dict
    params['location'] = f"{lat}, {lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
 # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("----------------------------------------")   
    
    time.sleep(1)


Retrieving Results for Index 0: ostrovnoy.
Closest hotel in ostrovnoy is Ostrovnoy.
----------------------------------------
Retrieving Results for Index 1: mahebourg.
Closest hotel in mahebourg is Mahebourg.
----------------------------------------
Retrieving Results for Index 2: qaanaaq.
Closest hotel in qaanaaq is Qaanaaq.
----------------------------------------
Retrieving Results for Index 3: zhuhai.
Closest hotel in zhuhai is Hohhot.
----------------------------------------
Retrieving Results for Index 4: cape town.
Closest hotel in cape town is Cape Town.
----------------------------------------
Retrieving Results for Index 5: atuona.
Closest hotel in atuona is Atuona.
----------------------------------------
Retrieving Results for Index 6: flinders.
Closest hotel in flinders is Wollongong.
----------------------------------------
Retrieving Results for Index 8: tiksi.
Closest hotel in tiksi is Tiksi.
----------------------------------------
Retrieving Results for Index 9: pushk

Retrieving Results for Index 68: beja.
Closest hotel in beja is Beja.
----------------------------------------
Retrieving Results for Index 69: fortuna.
Closest hotel in fortuna is Fortuna.
----------------------------------------
Retrieving Results for Index 70: ponta do sol.
Closest hotel in ponta do sol is Capitólio.
----------------------------------------
Retrieving Results for Index 71: qasigiannguit.
Closest hotel in qasigiannguit is Qasigiannguit.
----------------------------------------
Retrieving Results for Index 72: butaritari.
Closest hotel in butaritari is Ukiangang.
----------------------------------------
Retrieving Results for Index 73: virginia beach.
Closest hotel in virginia beach is Virginia Beach.
----------------------------------------
Retrieving Results for Index 74: hobyo.
Closest hotel in hobyo is Hobyo.
----------------------------------------
Retrieving Results for Index 75: saint george.
Closest hotel in saint george is Agios Georgios.
--------------------

Retrieving Results for Index 134: melfi.
Closest hotel in melfi is Melfi.
----------------------------------------
Retrieving Results for Index 135: kibaya.
Closest hotel in kibaya is Rukira.
----------------------------------------
Retrieving Results for Index 136: iqaluit.
Closest hotel in iqaluit is Iqaluit.
----------------------------------------
Retrieving Results for Index 137: lavrentiya.
Closest hotel in lavrentiya is Lavrentiya.
----------------------------------------
Retrieving Results for Index 138: morondava.
Closest hotel in morondava is Morondava.
----------------------------------------
Retrieving Results for Index 139: vaini.
Closest hotel in vaini is Jagalbet.
----------------------------------------
Retrieving Results for Index 140: georgetown.
Closest hotel in georgetown is Georgetown.
----------------------------------------
Retrieving Results for Index 141: sanchazi.
Closest hotel in sanchazi is Chengdu.
----------------------------------------
Retrieving Results

Closest hotel in kaabong is Kaabong.
----------------------------------------
Retrieving Results for Index 200: lazaro cardenas.
Closest hotel in lazaro cardenas is Lázaro Cárdenas.
----------------------------------------
Retrieving Results for Index 201: luanda.
Closest hotel in luanda is Luanda.
----------------------------------------
Retrieving Results for Index 202: rio gallegos.
Closest hotel in rio gallegos is Rio Gallegos.
----------------------------------------
Retrieving Results for Index 203: namatanai.
Closest hotel in namatanai is Namatanai.
----------------------------------------
Retrieving Results for Index 204: severo-kurilsk.
Closest hotel in severo-kurilsk is Severo-Kurilsk.
----------------------------------------
Retrieving Results for Index 205: hutang.
Closest hotel in hutang is Jinhua.
----------------------------------------
Retrieving Results for Index 206: namioka.
Closest hotel in namioka is Aomori.
----------------------------------------
Retrieving Resul

Closest hotel in saint-leu is Saint-Leu, Réunion.
----------------------------------------
Retrieving Results for Index 265: jalu.
Closest hotel in jalu is Jalu.
----------------------------------------
Retrieving Results for Index 266: alice springs.
Closest hotel in alice springs is Alice Springs.
----------------------------------------
Retrieving Results for Index 267: zaraza.
Closest hotel in zaraza is Zaraza.
----------------------------------------
Retrieving Results for Index 268: dingle.
Closest hotel in dingle is Pototan.
----------------------------------------
Retrieving Results for Index 269: bodden town.
Closest hotel in bodden town is Bodden Town.
----------------------------------------
Retrieving Results for Index 270: griffith.
Closest hotel in griffith is Griffith.
----------------------------------------
Retrieving Results for Index 271: coihaique.
Closest hotel in coihaique is Coyhaique.
----------------------------------------
Retrieving Results for Index 272: sao

Retrieving Results for Index 332: morant bay.
Closest hotel in morant bay is Morant Bay.
----------------------------------------
Retrieving Results for Index 333: raduzhnyy.
Closest hotel in raduzhnyy is Raduzhny.
----------------------------------------
Retrieving Results for Index 334: san patricio.
Closest hotel in san patricio is San Patricio.
----------------------------------------
Retrieving Results for Index 335: hulan ergi.
Closest hotel in hulan ergi is Qiqihar.
----------------------------------------
Retrieving Results for Index 336: thayetmyo.
Closest hotel in thayetmyo is Aunglan.
----------------------------------------
Retrieving Results for Index 337: acapulco.
Closest hotel in acapulco is Acapulco.
----------------------------------------
Retrieving Results for Index 338: port shepstone.
Closest hotel in port shepstone is Port Shepstone.
----------------------------------------
Retrieving Results for Index 339: tautira.
Closest hotel in tautira is Tautira.
----------

Retrieving Results for Index 397: tuatapere.
Closest hotel in tuatapere is Tuatapere.
----------------------------------------
Retrieving Results for Index 398: grindavik.
Closest hotel in grindavik is Grindavik.
----------------------------------------
Retrieving Results for Index 399: xining.
Closest hotel in xining is Xining.
----------------------------------------
Retrieving Results for Index 400: kieta.
Closest hotel in kieta is Kieta.
----------------------------------------
Retrieving Results for Index 401: kalabo.
Closest hotel in kalabo is Kalabo.
----------------------------------------
Retrieving Results for Index 402: sambava.
Closest hotel in sambava is Sambava.
----------------------------------------
Retrieving Results for Index 403: wawa.
Closest hotel in wawa is Wawa.
----------------------------------------
Retrieving Results for Index 404: manama.
Closest hotel in manama is Al Manama.
----------------------------------------
Retrieving Results for Index 405: labuan.

Retrieving Results for Index 464: atasu.
Closest hotel in atasu is Atasu.
----------------------------------------
Retrieving Results for Index 465: seshcha.
Closest hotel in seshcha is Seshcha.
----------------------------------------
Retrieving Results for Index 466: norilsk.
Closest hotel in norilsk is Norilsk.
----------------------------------------
Retrieving Results for Index 467: los teques.
Closest hotel in los teques is Los Teques.
----------------------------------------
Retrieving Results for Index 468: san cristobal.
Closest hotel in san cristobal is Quito.
----------------------------------------
Retrieving Results for Index 469: san ramon.
Closest hotel in san ramon is San Ramón.
----------------------------------------
Retrieving Results for Index 470: horta.
Closest hotel in horta is Barcelona.
----------------------------------------
Retrieving Results for Index 471: poum.
Closest hotel in poum is Delogozhdi.
----------------------------------------
Retrieving Results

Retrieving Results for Index 530: hirara.
Closest hotel in hirara is Hirara.
----------------------------------------
Retrieving Results for Index 531: russell.
Closest hotel in russell is Maipú.
----------------------------------------
Retrieving Results for Index 532: burns lake.
Closest hotel in burns lake is Burns Lake.
----------------------------------------
Retrieving Results for Index 533: hakkari.
Closest hotel in hakkari is Lempäälä.
----------------------------------------
Retrieving Results for Index 534: marzuq.
Closest hotel in marzuq is Al Hujayrah.
----------------------------------------
Retrieving Results for Index 535: villaviciosa.
Closest hotel in villaviciosa is Villaviciosa.
----------------------------------------
Retrieving Results for Index 536: padang.
Closest hotel in padang is Padang.
----------------------------------------
Retrieving Results for Index 537: matam.
Closest hotel in matam is Matam.
----------------------------------------
Retrieving Results 

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

TypeError: unsupported operand type(s) for -: 'str' and 'str'